In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_fresh.pkl', 'df_ohlcs.pkl', 'df_symlots.pkl',
       'df_und_margins.pkl', 'df_unds.pkl', 'dfrq.pkl', 'qopts.pkl',
       'z_new_qopts_temp.pkl'], dtype='<U20')

# Making price robust
* [ ] Prepare price contracts for stock and options
* [ ] Check on one price
* [ ] Check on optimium settings for price in executeAsync

## 1. One contract price

In [7]:
# imports
from engine import executeAsync, save_df, pre_process
from support import get_dte

In [8]:
# build a set of contracts to test
df = util.df(qopts.to_list()).iloc[:, 1:6].rename(columns={'lastTradeDateOrContractMonth': 'expiry'})

df['contract'] = qopts
df['dte'] = df.expiry.apply(get_dte)
df = df[df.dte>5].reset_index(drop=True)

In [30]:
# set the inputs
ct = df.sample(1).contract.iloc[0]

cts_small = df.sample(100).contract
cts_large =df.sample(250).contract

kwargs = {'FILL_DELAY': 7}

In [10]:
THIS_FOLDER = ''

In [11]:
# .Price
async def price(ib: IB, co, **kwargs) -> pd.DataFrame:
    """Optimal execAsync: CONCURRENT=40 and TIMEOUT=8 gives ~250 contract prices/min"""
    
    TEMPL_PATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "template", "df_price.pkl")
    df_empty = pd.read_pickle(TEMPL_PATH)
    cols = list(df_empty)

    try:
        FILL_DELAY = kwargs["FILL_DELAY"]
    except KeyError as ke:
        print(f"\nWarning: No FILL_DELAY supplied!. 5.5 second default is taken\n")
        FILL_DELAY = 5.5

    try:
        
        if isinstance(co, tuple):
            c = co[0]
        else:
            c = co
            
        df = util.df([c]).iloc[:, :6]\
                 .rename(columns={"lastTradeDateOrContractMonth": "expiry"})
        
    except (TypeError, AttributeError, ValueError) as err:
        print(f"\nError: contract {co} supplied is incorrect!"+\
              f"\n{err}"+\
              f"\n... and empty df will be returned !!!")
        
        df = df_empty
        
        return df # ! Aborted return with empty df for contract error
    
    tick = ib.reqMktData(c, genericTickList="106")

    await asyncio.sleep(FILL_DELAY)
    
    df = df.assign(localSymbol = c.localSymbol, contract = c)

    try:
        dfpr = util.df([tick])
        
        if dfpr.modelGreeks[0] is None:
            iv = dfpr.impliedVolatility
        else:
            iv = dfpr.modelGreeks[0].impliedVol
        
        df = df.assign(time = dfpr.time,
                       greeks = dfpr.modelGreeks,
                       bid = dfpr.bid,
                       ask = dfpr.ask,
                       close = dfpr['close'],
                       last = dfpr['last'],
                       price = dfpr["last"].combine_first(dfpr["close"]),
                       iv = iv)
        
    except AttributeError as e:
        
        print(f"\nError in {c.localSymbol}: {e}. df will have no price and iv!\n")
        
        df = df.assign(time = np.nan,
                       greeks = np.nan,
                       bid = np.nan,
                       ask = np.nan,
                       close = np.nan,
                       last = np.nan,
                       price = np.nan,
                       iv = np.nan)        
        
    ib.cancelMktData(c)

    return df

## 2. Handling multiple price contracts

In [25]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    dfps = ib.run(executeAsync(ib=ib,
                        algo=price,
                        cts=cts_small,
                        post_process=save_df,
                        CONCURRENT=40,
                        TIMEOUT=8,
                        OP_FILENAME="",
                        **kwargs,
                        ))


Done price for ['MXEF1210C1120.0', 'LYB0121P45.0'] 40 out of 100. Pending ['DVA1127C84.5', 'AMZN0416P3280.0']

Done price for ['ACN1211P155.0', 'HD0416C135.0'] 80 out of 100. Pending ['BLK1127P692.5', 'AMT0120P150.0']
Wall time: 23.3 s


In [26]:
dfps[dfps.price.isnull()]

,secType,conId,symbol,expiry,strike,right,localSymbol,contract,time,greeks,bid,ask,close,last,price,iv


In [33]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    dfps = ib.run(executeAsync(ib=ib,
                        algo=price,
                        cts=cts_large,
                        post_process=save_df,
                        CONCURRENT=40,
                        TIMEOUT=8,
                        OP_FILENAME="",
                        **kwargs,
                        ))


Done price for [] 0 out of 250. Pending ['CBOE0618P92.5', 'ISRG1218P410.0']

Done price for ['GOOG1204C1735.0', 'CSX1224P95.0'] 40 out of 250. Pending ['UPRO0319C59.0', 'NFLX1204P630.0']

Done price for ['UPRO0319C59.0'] 41 out of 250. Pending ['NFLX1204P630.0', 'CLX1224C255.0']

Done price for ['EOG0121P92.5', 'JNPR1224P28.0'] 80 out of 250. Pending ['CTXS1231C120.0', 'GD0521C160.0']

Done price for ['CTXS1231C120.0'] 81 out of 250. Pending ['GD0521C160.0', 'CMG0121C825.0']

Done price for ['ABC1218P100.0', 'CMI1218P220.0'] 120 out of 250. Pending ['LYB0115C35.0', 'GOOG1218C1445.0']

Done price for [] 120 out of 250. Pending ['LYB0115C35.0', 'GOOG1218C1445.0']

Done price for ['CHTR0115P725.0', 'OXY1204P5.0'] 160 out of 250. Pending ['HCA0319P95.0', 'ADBE1204P275.0']

Done price for ['HCA0319P95.0'] 161 out of 250. Pending ['ADBE1204P275.0', 'AZO1204P1147.5']

Done price for ['COST1211C270.0', 'KMX1127P85.0'] 200 out of 250. Pending ['UAA1211P9.0', 'XEO1204C1440.0']

Done price for [

In [34]:
dfps[dfps.price.isnull()]

,secType,conId,symbol,expiry,strike,right,localSymbol,contract,time,greeks,bid,ask,close,last,price,iv
